In [45]:
# Create a IAM user with programmatic access
# Attached the following policies to the user; 
# AmazonS3FullAccess
# AmazonRekognitionFullAccess


# Import the library for this project
from dotenv import load_dotenv
import boto3
import os
import cv2
import numpy as np

# Modules created by the team
import face_detect as fd, mask_detect as md


In [46]:
# Load .env enviroment variables
if load_dotenv():
    print(".env file loaded")
# Set keys from the .env file
access_key_id, secret_access_key = (os.getenv('AWS_KEY_ID'), os.getenv('AWS_SECRET'))

.env file loaded


In [47]:
# Define the Amazon Rekognition API client and input user credentials
client = boto3.client('rekognition',
                      aws_access_key_id = access_key_id,
                      aws_secret_access_key = secret_access_key,
                      region_name = 'us-west-2')

# Functions to work with Amazon Rekognition output
def AWS_Photo_Cloud_Check(photo_name="example_01.png", bucket='awstestphotos'):
    """Check the Picture for a face mask

    Args:
        photo_name (str, optional): The Name of the picture stored in the S3 Bucket. Defaults to "example_01.png".
        bucket (str, optional): The S3 Bucket to search for the photo. Defaults to 'awstestphotos'.

    Returns:
        [type]: The Json string describing the findings on the face.
    """
    try:
        response = client.detect_protective_equipment(Image={'S3Object':{'Bucket': bucket,'Name': photo_name }}, 
                    SummarizationAttributes={'MinConfidence':95, 'RequiredEquipmentTypes':['FACE_COVER']})
    except Exception as exn:
        print(exn)
    else:
        return response
def AWS_Photo_Check(photo_file_name="./Examples/example_01.png"):
    try:
        with open(photo_file_name, 'rb') as source_image:
            source_bytes = source_image.read()
        
        response = client.detect_protective_equipment(Image={'Bytes': source_bytes}, 
                SummarizationAttributes={'MinConfidence':80, 'RequiredEquipmentTypes':['FACE_COVER']})
    except Exception as exn:
        print(exn)
    else:
        return response
def Get_Pic_Summary_Info(response):
    return response['Summary']

def Get_Person_With_Mask(response):
    summary = Get_Pic_Summary_Info(response)
    return summary['PersonsWithRequiredEquipment']

def Get_Person_With_NO_Mask(response):
    summary = Get_Pic_Summary_Info(response)
    return summary['PersonsWithoutRequiredEquipment']

def Get_Person_Mask_State_Unknown(response):
    summary = Get_Pic_Summary_Info(response)
    return summary['PersonsIndeterminate']

def Get_Person(response, x):
    if type(x) == list and len(x)>0:
        x = x[0]
    for person in response['Persons']:
        if person['Id'] == x:
            return person
    return []
def Get_aws_Coordinate(coords=None):
    """Bounding Box
        {'BoundingBox': {'Width': 0.10592225193977356,
            'Height': 0.12687161564826965,
            'Left': 0.556527853012085,
            'Top': 0.22671788930892944}}
    """
    bb = coords
    if bb == None:
        return (None,None,None,None)
    else:
        w = bb['Width']
        h = bb['Height']
        l = bb['Left']
        t = bb['Top']
        return (w,h,l,t)
def Get_Face_Protection_Details_From_Person(person):
    # The person will contain the protected or unprotected 
    # Body Parts
    for parts in person['BodyParts']:
        if parts['Name'] == "FACE":
            face_confidence = parts['Confidence']
            # What equipment is on the face
            eq_detected = parts['EquipmentDetections']
            if eq_detected != []:
                face_protection =  eq_detected[0]['Type']
                face_cover_confidence = eq_detected[0]['Confidence']
                proper_face_cover = eq_detected[0]['CoversBodyPart']['Value']
                bounding_box = Get_aws_Coordinate(eq_detected[0]['BoundingBox'])
                return face_confidence, face_protection, face_cover_confidence, proper_face_cover, bounding_box
    return None, None, None, None, None
def Get_Face_Protection_Details_From_Person_Dict(person):
    conf_face, face_prot, face_prot_conf, face_prot_worn_prop, aws_bounding_box = Get_Face_Protection_Details_From_Person(person)
    return {
        'Confidence There Is A Face':conf_face
        , 'Type of Face Protection Found':face_prot
        , 'How confident there is a mask':face_prot_conf
        , 'Is the mask worn properly':face_prot_worn_prop
        , 'aws bounding box': aws_bounding_box
    }
# Get the Size of the Image
def Get_Image_Size(image):
    if image is None:
        print("There is NO Image!!")
    h, w, _ = image.shape
    return (h, w)
def Get_Image(image_file_name):
    print("Checking if the file exists?")
    if not os.path.exists(image_file_name):
        print(f"The File is missing {image_file_name}")
        return (None, False)
    print(f"The file was found: {image_file_name}")

    im = cv2.imread(filename=image_file_name)

    if im is None:
        print("Could NOT read the file! using LoadImage")
        im = cv2.cv.LoadImage(image_file_name)
    if im is None:
        print("Failed to get the image with LoadImage failing the process")
        return (None, False)
    else:
        return (im, True)
def Conv_2_Points(image, aws_bounding_box):
    if image is None or aws_bounding_box is None:
        print("Either there's no bounding box or there's NO Image detected!!!")

        return [
            (0,0)
            , (0,0)
            , (0,0)
            , (0,0)
            , (0,0)
        ], False

    WIDTH,HEIGHT,LEFT,TOP = 0,1,2,3
    h_size, w_size  = Get_Image_Size(image)
    left            = int(w_size * aws_bounding_box[LEFT])
    top             = int(h_size * aws_bounding_box[TOP])
    width           = int(w_size * aws_bounding_box[WIDTH])
    height          = int(h_size * aws_bounding_box[HEIGHT])
    points = [
        (left,top)
        , (left + width,top)
        , (left + width, top + height)
        , (left, top + height)
        , (left, top)
    ]
    return points, True
def Draw_Box(image, points, colour=(0,255,0), thick=3, linetype=8):
    if image is None:
        print("There was no image found !!!")
        return None

    for x in range(4):
        image = cv2.line(
            image
            , points[x], points[x+1]
            , color=colour
            , thickness=thick
            , lineType=linetype)

    return image
def Show_Image(image):
    if image is None:
        print("Image missing!!!")
        return
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [55]:

#AWS_Photo_Check
# Choose a photo that you wish to validate. 
# Make sure the photo is uploaded to the correct S3 bucket
#photo = 'example_01.png'  ############################ Edit This File Name
photo = "Giorgio.png"

In [56]:
file_info = fd.Face_Detect(f"./Examples/{photo}")

f = md.Mask_Detect()
mask_state, mask_confidence = f.Mask_Check(file_info.results[file_info.file_name][0][0])

print(f.Mask_Check(file_info.results[file_info.file_name][0][0]))

The file (./Examples/Giorgio.png) exists => True
Exiting Face Detect Class.
Exiting Mask Detect Class.
('Mask', 0.64489913)


In [57]:
# # Show the results
# Get_Pic_Summary_Info(response)
if mask_confidence<0.8:
    response = AWS_Photo_Cloud_Check(photo_name=photo)
    bp = Get_Person(response,0)
    a,b,c,d,e = Get_Face_Protection_Details_From_Person(bp)
    # # (a,b,c,d,e)
    # Get_Face_Protection_Details_From_Person_Dict(bp)
    fp = f"./Examples/{photo}"
    print(f"The file Path is:{fp}")
    im1, success = Get_Image(fp)
    # print(success)
    if success:
        print(e)
        points,_ = Conv_2_Points(im1,e)
    else:
        print("Sorry didn't get an image to work with!!!")
    print(Get_Face_Protection_Details_From_Person_Dict(bp))
    if success:
        im1 = Draw_Box(im1, points=points, colour=(0,255,0), thick=3,linetype=8)
    else:
        print("????????????")
    Show_Image(im1)
else:
    print("Above Confidence Level")

The file Path is:./Examples/Giorgio.png
Checking if the file exists?
The file was found: ./Examples/Giorgio.png
None
Either there's no bounding box or there's NO Image detected!!!
{'Confidence There Is A Face': None, 'Type of Face Protection Found': None, 'How confident there is a mask': None, 'Is the mask worn properly': None, 'aws bounding box': None}
